In [10]:
import os
import torch
import pandas as pd 
import numpy as np 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [11]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [14]:
# csv -> dataframe -> numpy array -> torch tensor
features = pd.read_csv("C:/Nesta/side projects/gambling_with_vscode/data/features.csv", index_col=0).to_numpy()

features = torch.from_numpy(features)

# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = features.to("cuda")

model = NeuralNetwork().to(device)
print(model)

logits = model(features)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")






NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=2, bias=True)
  )
)
